In [1]:
import pycbc
from pycbc.psd import interpolate, welch
from pycbc.filter import highpass_fir, lowpass_fir
from pycbc.types import TimeSeries
import bilby
from bilby.gw import detector
from pycbc.catalog import Merger
import numpy as np

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
# Function to whiten data
def whiten_data(ifo, event_name="GW170104"):
    h1 = Merger(event_name).strain(ifo) # load in data of certain event from certain interferometer
    
    h1 = highpass_fir(h1, 15, 8) # apply filter to remove lower frequencys

    psd = interpolate(welch(h1), 1.0 / h1.duration) # calculate the psd
    
    white_strain = (h1.to_frequencyseries() / psd ** 0.5).to_timeseries() #whitening the data

    smooth = highpass_fir(white_strain, 35, 8)
    smooth = lowpass_fir(smooth, 300, 8) # applying a low and high pass filter to further remove the noise

    if ifo == 'L1':
        smooth *= -1
        smooth.roll(int(.007 / smooth.delta_t))  # flip the L1 due to phase orientaion and apply time shift to accout for distance differene between h1 and l1

    return smooth

ifos = bilby.gw.detector.InterferometerList(["H1", "L1"]) # setting up interferometer list 

for ifo in ['H1', 'L1']:
    smooth_strain = whiten_data(ifo)
    ifo_object = ifos[0] if ifo == 'H1' else ifos[1]
    ifo_object.strain_data = smooth_strain # load in the whitened data for both inteferometers



In [13]:
delta_t = smooth_strain.sample_times[1] - smooth_strain.sample_times[0]
start_time = smooth_strain.sample_times[0]
duration = len(smooth_strain)*delta_t
sampling_frequency = 4096.0
minimum_frequency = 35

waveform_arguments = dict(
    waveform_approximant="IMRPhenomPv2",
    reference_frequency=50.0,
    minimum_frequency=minimum_frequency,
    
)

waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=4096.0,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,
)



ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=start_time,
)

# Set up priors
# priors = bilby.gw.prior.BBHPriorDict()
priors = dict(
    mass_1 = bilby.core.prior.Uniform(minimum = 28,maximum = 34),
    mass_2 = bilby.core.prior.Uniform(minimum = 18, maximum = 22),
    luminosity_distance = bilby.core.prior.Uniform(minimum = 950 , maximum = 1050),
    # geocent_time = bilby.core.prior.Uniform(minimum=0.0,maximum = 32.0), # time prior
    tilt_2 = bilby.core.prior.Sine(0, np.pi)
)


# Fix all other parameters to specific values
fixed_params = {
    "geocent_time": 0,  # fix merger time
    "phase": 0,  # fix phase
    "iota": np.pi/3,  # fix inclination angle
    "a_1": 0,  # fix spin magnitude 1
    "a_2": 0,  # fix spin magnitude 2
    "tilt_1": 0,  # fix spin tilt 1
    "ra": 1.0,  # fix right ascension
    "dec": 0.5,  # fix declination
    "psi": 0.1, 
    "theta_jn":0.5 # fix polarization angle
}

for key, value in fixed_params.items():
    priors[key] = value 

# priors["mass_1"] = bilby.core.prior.Uniform(minimum=20.0, maximum=45.0)  # mass 1 prior
# priors["mass_2"] = bilby.core.prior.Uniform(minimum=20.0, maximum=45.0)  #mass 2 prior 
# priors["luminosity_distance"] = bilby.core.prior.Uniform(minimum=1000.0, maximum=2500.0)  # distance prior

likelihood = bilby.gw.GravitationalWaveTransient( #setting up likelihood function
    interferometers=ifos, waveform_generator=waveform_generator
)

13:34 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [14]:
outdir = "outdir" # where data will be saved
label = "GW170104"
bilby.core.utils.setup_logger(outdir=outdir, label=label)

result = bilby.run_sampler(
    likelihood=likelihood, # running dynesty 
    priors=priors,
    sampler="dynesty",
    npoints=1,
    outdir=outdir,
    label=label,
)

result.plot_corner()

13:34 bilby INFO    : Running for label 'GW170104', output will be saved to 'outdir'
13:34 bilby INFO    : Using lal version 7.3.1
13:34 bilby INFO    : Using lal git version Branch: None;Tag: lalsuite-v7.19;Id: 84d780c102cf51ea1fdf7a1cbf0a626a5eca0d0a;;Builder: Duncan Macleod <duncan.macleod@ligo.org>;Repository status: CLEAN: All modifications committed
13:34 bilby INFO    : Using lalsimulation version 5.2.1
13:34 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsuite-v7.19;Id: 84d780c102cf51ea1fdf7a1cbf0a626a5eca0d0a;;Builder: Duncan Macleod <duncan.macleod@ligo.org>;Repository status: CLEAN: All modifications committed
/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/bilby/core/utils/log.py:73: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  vdict[key] = str(getattr(sys.modules[key], "__version__", "N/A"))
You should 

41it [1:27:17, ?it/s]

15:01 bilby INFO    : Run interrupted by signal 2: checkpoint and exit on 130
15:01 bilby INFO    : Written checkpoint file outdir/GW170104_resume.pickle



Exception while calling loglikelihood function:
  params: [ 32.69132869  18.88496365 998.329508     2.03178378]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/dynesty/dynesty.py", line 913, in __call__
    return self.func(np.asarray(x).copy(), *self.args, **self.kwargs)
  File "/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/bilby/core/sampler/dynesty.py", line 53, in _log_likelihood_wrapper
    return _sampling_convenience_dump.likelihood.log_likelihood_ratio()
  File "/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 420, in log_likelihood_ratio
    per_detector_snr = self.calculate_snrs(
  File "/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 275, in calculate_snrs
    signal = self._compute_full_waveform(
  File "/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 709, in _compute_full_waveform
    return interferometer.get_detector_response(signa

SystemExit: 130

/Users/maxrobertson/code/Grav Waves/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# need to make a dictionary of all the priors otherwise it does it for all 15